## DataFrame Exercises
In this notebook your job is to implement multiple small methods that process and analyze airtraffic data with DataFrames. DataFrames can be queried with SQL language and through SparkSQL API. Both of them can be used to implement methods in these exercises. The links below may be helpful:

- http://spark.apache.org/docs/latest/sql-programming-guide.html
- https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

We will use a sample of "2008.csv.bz2" which contains airtraffic data from https://dataverse.harvard.edu/api/access/datafile/1374917?gbrecs=true.

There are already two Spark SQL tables available from the start:

- table "carriers" inlcudes information of airlines
- table "airports" includes information of airports

In [1]:
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyspark.sql.functions as f
from pyspark.sql import Row, SparkSession

%matplotlib inline

spark = (
    SparkSession.builder.master("local[*]")
    .appName("main")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.shuffle.service.enabled", "true")
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# names of tables
airTraffic = "airtraffic"
carriers = "carriers"
airports = "airports"

carriersTable = spark.read.csv("carriers.csv", inferSchema="true", header="true")
carriersTable.createOrReplaceTempView(carriers)

airportsTable = spark.read.csv("airports.csv", inferSchema="true", header="true")
airportsTable.createOrReplaceTempView(airports)

In [3]:
# Methods and variables that will be used in more than one tests

# Test if arrays that contain Row are equal
def correctRows(testArray, correctArray):
    for i in range(0, len(correctArray)):
        assert (
            testArray[i].asDict() == correctArray[i].asDict()
        ), "the row was expected to be %s but it was %s" % (
            correctArray[i].asDict(),
            testArray[i].asDict(),
        )


# Path of smaller airtraffic data set
sampleFile = "2008_sample.csv"
testFile = "2008_testsample.csv"

## Load Data and Register 
`loadDataAndRegister` loads airtraffic data and registers it as a table so that we can use it later for Spark SQL. 

param `path`: path of file that should be loaded and registered.

`return`: DataFrame containing airtraffic information.

The schema of returned DataFrame should be:

Name | Type
------| :-----
Year  | integer (nullable = true)
Month | integer (nullable = true)
DayofMonth | integer (nullable = true)
DayOfWeek | integer (nullable = true)
DepTime | integer (nullable = true)
CRSDepTime | integer (nullable = true)
ArrTime | integer (nullable = true)
CRSArrTime | integer (nullable = true)
UniqueCarrier | string (nullable = true)
FlightNum | integer (nullable = true)
TailNum | string (nullable = true)
ActualElapsedTime | integer (nullable = true)
CRSElapsedTime | integer (nullable = true)
AirTime | integer (nullable = true)
ArrDelay | integer (nullable = true)
DepDelay | integer (nullable = true)
Origin | string (nullable = true)
Dest | string (nullable = true)
Distance | integer (nullable = true)
TaxiIn | integer (nullable = true)
TaxiOut | integer (nullable = true)
Cancelled | integer (nullable = true)
CancellationCode | string (nullable = true)
Diverted | integer (nullable = true)
CarrierDelay | integer (nullable = true)
WeatherDelay | integer (nullable = true)
NASDelay | integer (nullable = true)
SecurityDelay | integer (nullable = true)
LateAircraftDelay | integer (nullable = true)

Hints:
- How to load csv data: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader
- If you just load data using `inferSchema="true"`, some of the fields which shoud be Integers are casted to Strings because null values are represented as "NA" strings in the data. E.g. 2008,7,2,3,733,735,858,852,DL,1551,N957DL,85,77,42,6,-2,CAE, ATL,191,15,28,0,,0,NA,NA,NA,NA,NA. Therefore you need to replace all "NA" strings with null. Option "nullValue" is helpful.
- Please use the variable `airTraffic` as table name.

In [4]:
def loadDataAndRegister(path):

    spark.read.csv(
        path, inferSchema="true", header="true", nullValue="NA"
    ).createOrReplaceTempView(airTraffic)

    return spark.sql("select * from airtraffic")
    raise NotImplementedError()

In [5]:
# example print

data = loadDataAndRegister(testFile)
print(data.toPandas().head(5))
data.schema

21/10/03 10:23:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


   Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  ArrTime  \
0  2008      1           4          7      632         615      756   
1  2008      5           5          1      630         615      741   
2  2008      5           6          2      611         615      729   
3  2008      5           7          3      611         615      725   
4  2008      3           8          4      613         615      742   

   CRSArrTime UniqueCarrier  FlightNum  ... TaxiIn  TaxiOut  Cancelled  \
0         735           02Q       4794  ...      8       14          1   
1         735            EV       4794  ...      8        7          1   
2         735            EV       4794  ...      9       11          0   
3         735            EV       4794  ...      6        7          0   
4         735            EV       4794  ...      6       25          0   

   CancellationCode  Diverted  CarrierDelay WeatherDelay NASDelay  \
0                 D         0          17.0          0.0   

StructType(List(StructField(Year,IntegerType,true),StructField(Month,IntegerType,true),StructField(DayofMonth,IntegerType,true),StructField(DayOfWeek,IntegerType,true),StructField(DepTime,IntegerType,true),StructField(CRSDepTime,IntegerType,true),StructField(ArrTime,IntegerType,true),StructField(CRSArrTime,IntegerType,true),StructField(UniqueCarrier,StringType,true),StructField(FlightNum,IntegerType,true),StructField(TailNum,StringType,true),StructField(ActualElapsedTime,IntegerType,true),StructField(CRSElapsedTime,IntegerType,true),StructField(AirTime,IntegerType,true),StructField(ArrDelay,IntegerType,true),StructField(DepDelay,IntegerType,true),StructField(Origin,StringType,true),StructField(Dest,StringType,true),StructField(Distance,IntegerType,true),StructField(TaxiIn,IntegerType,true),StructField(TaxiOut,IntegerType,true),StructField(Cancelled,IntegerType,true),StructField(CancellationCode,StringType,true),StructField(Diverted,IntegerType,true),StructField(CarrierDelay,IntegerType

In [6]:
"""loadDataAndRegister tests"""

df = loadDataAndRegister(testFile)

# Table "airtraffic" should exists
assert (
    spark.sql("SHOW TABLES Like 'airtraffic'").count() == 1
), "there was expected to be a table called 'airtraffic'"

# Columns should have correct values
third = df.collect()[2]
correctRow = Row(
    Year=2008,
    Month=5,
    DayofMonth=6,
    DayOfWeek=2,
    DepTime=611,
    CRSDepTime=615,
    ArrTime=729,
    CRSArrTime=735,
    UniqueCarrier="EV",
    FlightNum=4794,
    TailNum="N916EV",
    ActualElapsedTime=78,
    CRSElapsedTime=80,
    AirTime=58,
    ArrDelay=-6,
    DepDelay=-4,
    Origin="ROA",
    Dest="ATL",
    Distance=357,
    TaxiIn=9,
    TaxiOut=11,
    Cancelled=0,
    CancellationCode=None,
    Diverted=0,
    CarrierDelay=None,
    WeatherDelay=None,
    NASDelay=None,
    SecurityDelay=None,
    LateAircraftDelay=None,
).asDict()

assert third.asDict() == correctRow, "the row was expected to be %s but it was %s" % (
    correctRow,
    third.asDict(),
)

## Flight Count
`flightCount` gets the number of flights for each airplane. The "TailNum" column is unique for each airplane so it can be used.

param `df`: Airtraffic DataFrame created using `loadDataAndRegister`.

`return`: DataFrame containing number of flights per TailNum. DataFrame should include columns "TailNum" and "count" (the number of flights for an airplane) . Airplanes whose TailNum is null should not be included in the returned DataFrame. **The returned DataFrame should be sorted by count in descending order.** 

Example output:

TailNum|count
-------:|-----
N693BR| 1526|
N646BR| 1505|
N476HA| 1490|
N485HA| 1441|
N486HA| 1439|


In [7]:
def flightCount(df):
    return (
        df.filter(df.TailNum.isNotNull())
        .groupBy("TailNum")
        .count()
        .orderBy("count", ascending=False)
    )
    raise NotImplementedError()

In [8]:
# example print
data = loadDataAndRegister(sampleFile)
flightCount(data).show(5)

+-------+-----+
|TailNum|count|
+-------+-----+
| N317AE|    2|
| N515MJ|    2|
| N625CA|    1|
| N27213|    1|
| N932AE|    1|
+-------+-----+
only showing top 5 rows



In [9]:
"""flightCount tests"""

data = loadDataAndRegister(testFile)

correct = [
    Row(TailNum="N824AS", count=2),
    Row(TailNum="N856AS", count=1),
    Row(TailNum="N886AS", count=1),
    Row(TailNum="N916EV", count=1),
    Row(TailNum="N873AS", count=1),
    Row(TailNum="N881AS", count=1),
]

correctRows(flightCount(data).collect(), correct)

AssertionError: the row was expected to be {'TailNum': 'N856AS', 'count': 1} but it was {'TailNum': 'N873AS', 'count': 1}

### You can either use Spark SQL or normal DataFrame (given as parameter) transformations to implement the methods below.

## Cancelled Due to Security
`cancelledDueToSecurity` finds which flights were cancelled due to security reasons. 

`return`: DataFrame containing flights which were cancelled due to security reasons (CancellationCode = "D"). Columns "FlightNum" and "Dest" should be included.

Example output:

FlightNum|Dest|
----:|-------
4285| DHN|
4790| ATL|
3631| LEX|
3632| DFW|

In [ ]:
def cancelledDueToSecurity(df):
    return df.filter(df.CancellationCode == "D").select(["FlightNum", "Dest"])
    raise NotImplementedError()

In [ ]:
# example print

data = loadDataAndRegister(sampleFile)
cancelledDueToSecurity(data).show(5)

In [ ]:
"""cancelledDueToSecurity tests"""

data = loadDataAndRegister(testFile)
correct = [Row(FlightNum=4794, Dest="JFK"), Row(FlightNum=4794, Dest="ATL")]
correctRows(cancelledDueToSecurity(data).collect(), correct)

## Longest Weather Delay
`longestWeatherDelay` finds the longest weather delay between January and March (1.1-31.3).

`return`: DataFrame containing the longest weather delay.

Example output:

|_c0|
|-------:|
|1148|

In [ ]:
def longestWeatherDelay(df):
    return (
        df.filter((df.Month >= 1) & (df.Month <= 3))
        .select("WeatherDelay")
        .agg({"WeatherDelay": "max"})
    )
    raise NotImplementedError()

In [ ]:
# example print

data = loadDataAndRegister(sampleFile)
longestWeatherDelay(data).show()

In [ ]:
"""longestWeatherDelay tests"""

data = loadDataAndRegister(testFile)
test = longestWeatherDelay(data).first()[0]

assert test == 7, "the longest weather delay was expected to be 7 but it was %s" % test

## Did Not Fly
`didNotFly` finds which airlines didn't have flights. 

`return`: DataFrame containig descriptions (names) of airlines that didn't have flights.

Example output:

|         Description|
|--------------------|
|Aero Transcolombiana|
|Transmeridian Air...|
|Luftransport-Unte...|
|Euro Atlantic Air...|
|    Pearson Aircraft|


Hints:
- Schema "UniqueCarrier" (the code of airline) of table "airtraffic" can be used when implementing this method.
- Table "carriers" containing airlines' names is already loaded to "carriersTable" object at the beginning.

In [ ]:
def didNotFly(df):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example print

data = loadDataAndRegister(sampleFile)
didNotFly(data).show(5)

In [ ]:
"""didNotFly tests"""

data = loadDataAndRegister(testFile)
test = didNotFly(data).count()

assert test == 1489, (
    "the amount of airlines that didn't fly was expected to be 1489 but it was %s"
    % test
)

## Flights from Vegas to JFK
`flightsFromVegasToJFK` finds airlines that fly from Vegas to JFK.

`return`: DataFrame containing columns "Descriptions" (names of airlines) and "Num" (number of flights). **The DataFrame should be sorted by Num in descending order.**

Example output:

|         Description|Num|
|--------------------|---|
|     JetBlue Airways|566|
|Delta Air Lines Inc.|441|
|US Airways Inc. (...|344|
|American Airlines...|121|

Hints:
- Vegas iasa code: LAS. JFK iasa code: JFK

In [ ]:
def flightsFromVegasToJFK(df):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example print

data = loadDataAndRegister(sampleFile)
flightsFromVegasToJFK(data).show(5)

In [ ]:
"""flightsFromVegasToJFK tests"""

data = loadDataAndRegister(testFile)
correct = [
    Row(Description="Titan Airways", Num=1),
    Row(Description="Atlantic Southeast Airlines", Num=1),
]
correctRows(flightsFromVegasToJFK(data).collect(), correct)

## Time Spent in Taxiing
`timeSpentTaxiing` finds how much time airplanes spent in moving from gate to the runway and vise versa at an airport on average. 

`return`: DataFrame contains the average time spent in taxiing per airport. The DataFrame should contain columns "airport" (iata codes of airports) and "taxi" (the average time spent in taxiing). **The DataFrame should be sorted by "taxi" in ascending order.**

Example output:

|airport|             taxi|
|-------|-----------------|
|    DLG|              4.0|
|    BRW|5.051010191310567|
|    OME|6.034800675790983|
|    AKN|             6.75|
|    SCC|6.842553191489362|

Hints:
- Columns "TaxiIn" and "TaxiOut" tells time spend in taxiing. "TaxiIn" means time spent in taxiing in departure ("Origin") airport and "TaxiOut" spent in taxiing in arrival ("Dest") airport. The wanted average is (average taxiing at origin for a given destination + average taxiing at destination for a given matching origin) / 2.

In [ ]:
def timeSpentTaxiing(df):

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example print

data = loadDataAndRegister(sampleFile)
timeSpentTaxiing(data).show(5)

In [ ]:
"""timeSpentTaxiing tests"""

data = loadDataAndRegister(testFile)
correct = [Row(airport="LAS", taxi=11.0), Row(airport="JFK", taxi=13.25)]
correctRows(timeSpentTaxiing(data).collect(), correct)

## Distance Median
`distanceMedian` finds the median travel distance.

`return`: DataFrame containing the median travel distance.

Example output:

|_ c0|
|---|
|583.0|

Hints:
- Schema "Distance" of table "airtraffic" contains distance information.
- You should use exact percentile functions like Spark SQL build-in [percentile function](https://spark.apache.org/docs/latest/api/sql/index.html#percentile).  
- What does percentile mean? Please check: https://en.wikipedia.org/wiki/Percentile#Third_variant and http://onlinestatbook.com/2/introduction/percentiles.html

In [ ]:
def distanceMedian(df):

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example print

data = loadDataAndRegister(sampleFile)
distanceMedian(data).show()

In [ ]:
"""distanceMedian tests"""

data = loadDataAndRegister(testFile)
test = distanceMedian(data).first()[0]
assert test == 357.0, (
    "the distance median was expected to be 357.0 but it was %s" % test
)

## Score95
`score95` finds the percentile, below which 95% of the delay (CarrierDelay) observations may be found. 

return: DataFrame containing the 95th percentile of carrier delay. 

Example output:

|_ c0|
|----|
|77.0|

Hints:
- You should use exact percentile functions like Spark SQL build-in [percentile function](https://spark.apache.org/docs/latest/api/sql/index.html#percentile). 

In [ ]:
def score95(df):

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def score95(df):

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
"""score95 tests"""

data = loadDataAndRegister(testFile)
test = score95(data).first()[0]
assert test == 17.0, "the score95 was expected to be 17.0 but it was %s" % test

## Cancelled Flights
`cancelledFlights` finds airports where flights were cancelled. 

return: DataFrame containing columns "airport", "city" and "percentage". 
- Columns "airport" and "city" can be found from table "airports". Column "percentage" is the cancellation percentage of each airport (number of cancelled flights/total of flights).
- **The returned DataFrame should be sorted by "percentage" and secondly by "airport" both in descending order.**

Example output:

|             airport|       city|         percentage|
|--------------------|-----------|-------------------|
|Pellston Regional...|   Pellston| 0.3157894736842105|
|  Waterloo Municipal|   Waterloo|               0.25|
|  Telluride Regional|  Telluride|0.21084337349397592|
|Houghton County M...|    Hancock|0.19834710743801653|
|Rhinelander-Oneid...|Rhinelander|            0.15625|

In [ ]:
def cancelledFlights(df):

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example print

data = loadDataAndRegister(sampleFile)
cancelledFlights(data).show(5)

In [ ]:
"""cancelledFlights tests"""

data = loadDataAndRegister(testFile)
correct = [
    Row(airport="McCarran International", city="Las Vegas", percentage=0.5),
    Row(airport="Roanoke Regional/ Woodrum ", city="Roanoke", percentage=0.25),
]
correctRows(cancelledFlights(data).collect(), correct)

## Least Squares
`leastSquares` calculates the [linear least squares](https://en.wikipedia.org/wiki/Linear_least_squares) approximation for relationship between DepDelay and WeatherDelay (y=bx+c, where x represents DepDelay and y WeatherDelay, b is the slope and c constant term). We want to predict WeatherDelay.

`return`: tuple that has the constant term first and the slope second. If least squares can not be calculated, return 0.0 as terms.

Hints:
- Filter out entries where DepDelay<0 before calculating the linear least squares.
- There are definitely multiple datapoints for a single DepDelay value so calculate the average WeatherDelay per DepDelay.
- These links may be helpful:
    - https://en.wikipedia.org/wiki/Simple_linear_regression#Fitting_the_regression_line
    - http://www.neoprogrammics.com/linear_least_squares_regression
    - https://www.youtube.com/watch?v=JvS2triCgOY

In [ ]:
def leastSquares(df):

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# example print

data = loadDataAndRegister(sampleFile)
leastSquares(data)

In [ ]:
data = loadDataAndRegister(testFile)
test = leastSquares(data)
assert test == (952.0, -56.0), (
    "the answer was expected to be (952.0, -56.0) but it was %s" % test
)

In [ ]:
spark.stop()